In [ ]:
! git clone --quiet https://github.com/clovaai/donut.git
# ! cd donut && pip install .

In [ ]:
#https://github.com/clovaai/donut/issues/207 #linea 101
! sed -i '101d' /content/donut/donut/model.py

In [ ]:
! pip install --quiet --no-cache-dir transformers==4.25.1 sconf zss datasets[vision] sentencepiece pytorch-lightning==2.0.7 timm==0.5.4
#timm==0.9.10
# #--quiet
# #nooo# ! pip install donut-python gradio
# ! pip install transformers==4.29.2
# #nooo timm==0.5.4
# #noooo ! pip install pytorch-lightning==1.6.4
# ! pip install sconf
# ! pip install zss
# ! pip install datasets[vision]
# ! pip install sentencepiece
# ! pip install timm==0.9.10
# ! pip install timm==0.6.5
# ! pip install timm==0.5.4
# # ! git lfs install
# # ! git clone https://huggingface.co/spaces/naver-clova-ix/donut-base-finetuned-cord-v2
# # ! cd donut-base-finetuned-cord-v2 && pip install -r requirements.txt
# ! pip install pytorch-lightning==2.0.7

In [ ]:
# from donut import DonutModel
# from PIL import Image
# import torch
# model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")#, ignore_mismatched_sizes=True)
# if torch.cuda.is_available():
#     model.half()
#     device = torch.device("cuda")
#     model.to(device)
# else:
#     model.encoder.to(torch.bfloat16)
# model.eval()
# image = Image.open("/content/donut-base-finetuned-cord-v2/sample_image_cord_test_receipt_00004.png").convert("RGB")
# output = model.inference(image=image, prompt="<s_cord-v2>")
# output

In [ ]:
# ! gdown --fuzzy https://drive.google.com/file/d/1rgCwufs0IVtTYBk5BSmqEHUe_MKFRhJI/view?usp=sharing
# ! unzip -oq donut_dataset_elecciones.zip
# ! rm donut_dataset_elecciones.zip
# ! mv /content/data/donut_dataset /content/data/elecciones_generales

In [ ]:
! rm -rf /content/data/
! unzip -oq donut_dataset_elecciones.zip
# ! rm donut_dataset_elecciones.zip
! mv /content/data/donut_dataset /content/data/elecciones_generales

In [ ]:
#convert tiff files to jpeg

from glob import glob
from tqdm.auto import tqdm
from PIL import Image as ImagePil
from os import path, remove

! du -sh /content/data/elecciones_generales
train_images_counter = 0

for tiff_path in tqdm(glob('/content/data/elecciones_generales/**/*.tiff')):
  if 'train' in tiff_path: train_images_counter += 1
  tiff_path_dir, tiff_filename = path.split(path.abspath(tiff_path))
  tiff_filename_wo_ext = path.splitext(tiff_filename)[0]
  jpeg_path = path.join(tiff_path_dir, tiff_filename_wo_ext + '.jpg')
  ImagePil.open(tiff_path).convert('RGB').save(jpeg_path, quality=90, subsampling=0, optimize=False) #subsampling=1, optimize=True
  remove(tiff_path)

! du -sh /content/data/elecciones_generales

In [ ]:
TRAIN_BATCH_SIZES = 2
MAX_EPOCHS = 20
WARMUP_STEPS = int((train_images_counter /  TRAIN_BATCH_SIZES) * MAX_EPOCHS * .1) #10% of total training steps

In [ ]:
%%bash -s "$train_images_counter" "$TRAIN_BATCH_SIZES" "$MAX_EPOCHS" "$WARMUP_STEPS"

cat >/content/donut/config/train_generales.yaml<<EOL
resume_from_checkpoint_path: null # only used for resume_from_checkpoint option in PL
result_path: "./result"
pretrained_model_name_or_path: "naver-clova-ix/donut-base" # loading a pre-trained model (from moldehub or path)
dataset_name_or_paths: ["/content/data/elecciones_generales"] # loading datasets (from moldehub or path)
sort_json_key: False # cord dataset is preprocessed, and publicly available at https://huggingface.co/datasets/naver-clova-ix/cord-v2
train_batch_sizes: [$2]
val_batch_sizes: [$2]
# input_size: [704, 320] # when the input resolution differs from the pre-training setting, some weights will be newly initialized (but the model training would be okay)
# input_size: [1280, 960]
input_size: [640, 320]
max_length: 256
align_long_axis: False
num_nodes: 1
seed: 2022
lr: 3e-5
warmup_steps: $4 # 10% of total steps, equals to num_training_samples_per_epoch / train_batch_sizes * max_epochs / 10
num_training_samples_per_epoch: $1 # Set it to the number of training images you have
max_epochs: $3 #30
max_steps: -1
num_workers: 2 #8???
val_check_interval: 1.0
check_val_every_n_epoch: 3 # 5 #10
gradient_clip_val: 1.0
verbose: True
EOL

In [ ]:
! cd /content/donut && python train.py --config /content/donut/config/train_generales.yaml

In [ ]:
! cd /content/donut && python test.py --dataset_name_or_path /content/data/elecciones_generales --pretrained_model_name_or_path /content/donut/result/train_generales/20231114_111917 --save_path ./result/output.json